In [1]:
from datetime import datetime, timedelta  # Import timedelta from datetime
from GoogleNews import GoogleNews
import pandas as pd

Fin-Bert

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
model = pipeline("text-classification", model="ProsusAI/finbert")

c:\Users\Ishit\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data preparation

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

c:\Users\Ishit\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import torch
from torch import nn

# def classify_sentiment(text_list):
#     pt_batch = tokenizer(
#         text_list,
#         padding=True,
#         truncation=True,
#         max_length=512,
#         return_tensors="pt"
#     )

#     pt_outputs = model(**pt_batch)
#     pt_predictions = nn.functional.softmax(pt_outputs.logits, dim = 1)

#     max_prediction = torch.argmax(pt_predictions, dim=1).tolist()
#     max_scores = torch.max(pt_predictions, dim=1).values.tolist()

#     # print(max_prediction)
#     for i in range(len(pt_outputs.logits)):
#         if max_prediction[i] == 2:
#             max_prediction[i] = 0 if pt_predictions[i][0] > pt_predictions[i][1] else 1
#             max_scores[i] = pt_predictions[i][0] if pt_predictions[i][0] > pt_predictions[i][1] else pt_predictions[i][1]
#     # map 0 to positive, 1 to negative in max_prediction

#     return max_prediction, max_scores

import torch
from torch import nn


# def classify_sentiment(text_list):
#     pt_batch = tokenizer(
#         text_list,
#         padding=True,
#         truncation=True,
#         max_length=512,
#         return_tensors="pt"
#     )

#     pt_outputs = model(**pt_batch)
#     pt_predictions = nn.functional.softmax(pt_outputs.logits, dim = 1)

#     max_prediction = torch.argmax(pt_predictions, dim=1).tolist()
#     max_scores = torch.max(pt_predictions, dim=1).values.tolist()

#     # print(max_prediction)
#     for i in range(len(pt_outputs.logits)):
#         if max_prediction[i] == 2:
#             max_prediction[i] = 0 if pt_predictions[i][0] > pt_predictions[i][1] else 1
#             max_scores[i] = pt_predictions[i][0] if pt_predictions[i][0] > pt_predictions[i][1] else pt_predictions[i][1]
#     # map 0 to positive, 1 to negative in max_prediction

#     return max_prediction, max_scores

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

def scrape_news(company, pages = 1):

    # URL of the website to scrape
    columns = ['datetime', 'title', 'source', 'link']
    df = pd.DataFrame(columns=columns)

    for page in range(1, pages+1):
        url = f'https://markets.businessinsider.com/news/{company}-stock?p={page}'

        
        response = requests.get(url)
        html = response.text
        soup = BeautifulSoup(html, 'lxml')

        articles = soup.find_all('div', class_='latest-news__story')

        for article in articles:
            datetime = article.find('time', class_='latest-news__date').get('datetime')
            title = article.find('a', class_='news-link').text
            source = article.find('span', class_='latest-news__source').text
            link = article.find('a', class_='news-link').get('href')

            df = pd.concat([pd.DataFrame([[datetime, title, source, link]], columns=df.columns), df], ignore_index=True)

            
    return df


In [127]:
# a = soup.find_all('div', class_='latest-news__story')
# len(a)

In [4]:
df = scrape_news('aapl',300)

In [5]:
len(df)

15000

In [6]:
# remove time from datetime
df['datetime'] = pd.to_datetime(df['datetime']).dt.date
df.set_index('datetime', inplace=True)

C:\Users\Ishit\AppData\Local\Temp\ipykernel_21888\3893246076.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['datetime'] = pd.to_datetime(df['datetime']).dt.date


In [150]:
# df['sentiment'] = df['title'].apply(lambda x: model(x)[0]['label'])
# df['score'] = df['title'].apply(lambda x: model(x)[0]['score'])

In [7]:
# find missing dates
missing_dates = pd.date_range(start=df.index.min(), end=df.index.max()).difference(df.index)

In [8]:
missing_dates

DatetimeIndex(['2018-05-20', '2018-05-28', '2018-06-03', '2018-06-07',
               '2018-06-10', '2018-06-24', '2018-07-01', '2018-07-04',
               '2018-07-06', '2018-07-07',
               ...
               '2024-08-30', '2024-08-31', '2024-09-01', '2024-09-02',
               '2024-09-07', '2024-09-08', '2024-09-14', '2024-09-15',
               '2024-09-28', '2024-09-29'],
              dtype='datetime64[ns]', length=302, freq=None)

In [9]:
df.source.unique()

array(['Business Insider', 'Reuters', 'RTTNews', 'PR Newswire',
       'Markets Insider Automation', 'Premium', 'Moodys', 'InvestorPlace',
       'Seeking Alpha', 'Benzinga', 'TipRanks', 'GlobeNewswire'],
      dtype=object)

In [10]:
df.index.min(), df.index.max()

(datetime.date(2018, 5, 17), datetime.date(2024, 10, 4))

In [11]:
df.head()

,title,source,link
datetime,,,
2018-05-17,The rapid-fire way Steve Jobs got employees to...,Business Insider,http://www.businessinsider.com/steve-jobs-pixa...
2018-05-18,Apple pays Ireland first tranche of disputed t...,Reuters,http://feeds.reuters.com/~r/reuters/companyNew...
2018-05-18,CORRECTED-Apple pays Ireland first tranche of ...,Reuters,http://feeds.reuters.com/~r/reuters/companyNew...
2018-05-19,Apple fans are raging that the new MacBook Pro...,Business Insider,http://www.businessinsider.com/apple-macbook-p...
2018-05-21,Apple just cut the price of the one cable you ...,Business Insider,http://www.businessinsider.com/apple-usb-c-to-...


In [157]:
# mode_sentiment = df.groupby(df.index)['sentiment'].agg(lambda x: x.mode().tolist())

In [12]:
# mode_sentiment.value_counts()

In [12]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [13]:
import torch
from torch import nn

def classify_sentiment(text_list, batch_size=32):
    all_predictions = []
    all_scores = []

    # Process in batches
    for i in range(0, len(text_list), batch_size):
        pt_batch = tokenizer(
            text_list[i:i + batch_size],  # Get a batch of texts
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )

        with torch.no_grad():  # Disable gradient calculation
            pt_outputs = model(**pt_batch)
            pt_predictions = nn.functional.softmax(pt_outputs.logits, dim=1)

        max_prediction = torch.argmax(pt_predictions, dim=1).tolist()
        max_scores = torch.max(pt_predictions, dim=1).values.tolist()

        # Adjust predictions and scores
        for j in range(len(max_prediction)):
            if max_prediction[j] == 2:  # Check for neutral sentiment
                max_prediction[j] = 0 if pt_predictions[j][0] > pt_predictions[j][1] else 1
                max_scores[j] = pt_predictions[j][0] if pt_predictions[j][0] > pt_predictions[j][1] else pt_predictions[j][1]

        # Extend results
        all_predictions.extend(max_prediction)
        all_scores.extend(max_scores)

    return all_predictions, all_scores




In [14]:
x,y = classify_sentiment(["high earnings","low"])

In [15]:
detached_tensors = [t.detach().item() for t in y]
detached_tensors

[0.3832497000694275, 0.129344642162323]

In [16]:
sentiment, scores = classify_sentiment(df['title'].tolist())

In [17]:
df['sentiment'] = sentiment
df['scores']  = [t.detach().item() if isinstance(t, torch.Tensor) else t for t in scores]

In [38]:
# df['sentiment'] = x
# scores

In [18]:
df['scores'][-150:]

datetime
2024-08-09    0.150890
2024-08-09    0.854619
2024-08-09    0.870183
2024-08-09    0.160017
2024-08-10    0.590021
                ...   
2024-10-01    0.104611
2024-10-02    0.944245
2024-10-02    0.947279
2024-10-03    0.743496
2024-10-04    0.227229
Name: scores, Length: 150, dtype: float64

In [19]:
mapping = {0:'positive',1:'negative'}
df['sentiment'] = df['sentiment'].map(mapping)

In [20]:
df.head()

,title,source,link,sentiment,scores
datetime,,,,,
2018-05-17,The rapid-fire way Steve Jobs got employees to...,Business Insider,http://www.businessinsider.com/steve-jobs-pixa...,negative,0.074973
2018-05-18,Apple pays Ireland first tranche of disputed t...,Reuters,http://feeds.reuters.com/~r/reuters/companyNew...,positive,0.056335
2018-05-18,CORRECTED-Apple pays Ireland first tranche of ...,Reuters,http://feeds.reuters.com/~r/reuters/companyNew...,positive,0.073576
2018-05-19,Apple fans are raging that the new MacBook Pro...,Business Insider,http://www.businessinsider.com/apple-macbook-p...,negative,0.703944
2018-05-21,Apple just cut the price of the one cable you ...,Business Insider,http://www.businessinsider.com/apple-usb-c-to-...,negative,0.324263


In [104]:
filtered_df = df[df.index == pd.to_datetime('2024-09-9').date() ]
filtered_df.head()

,title,source,link,sentiment,scores
datetime,,,,,
2024-09-09,Apple shows off AI-packed iPhone 16 line,Seeking Alpha,https://seekingalpha.com/news/4147993-apple-ip...,positive,0.208388
2024-09-09,"Schwab Trading Activity gauge slips in August,...",Seeking Alpha,https://seekingalpha.com/news/4148111-schwab-t...,negative,0.971948
2024-09-09,Apple Introduces Watch Series 10 With Thinner ...,RTTNews,/news/stocks/apple-introduces-watch-series-10-...,positive,0.166436
2024-09-09,Apple Unveils AirPods 4,RTTNews,/news/stocks/apple-unveils-airpods-4-1033758502,positive,0.105550
2024-09-09,"Apple Launches Next-Gen IPhone 16 Pro, IPhone ...",RTTNews,/news/stocks/apple-launches-next-gen-iphone-16...,positive,0.200915


In [86]:
len(filtered_df["sentiment"].mode())

0

In [106]:
filtered_df["sentiment"].mode()[0]
# pos_score = filtered_df[ filtered_df['sentiment'] == 'positive']["scores"].sum()
# neg_score = filtered_df[ filtered_df['sentiment'] == 'negative']["scores"].sum()

'positive'

In [73]:
neg_score = filtered_df[ filtered_df['sentiment'] == 'negative']["scores"].sum()

0.9046845436096191

In [56]:
pd.to_datetime('2024-07-05').date() == df.index.min()

True

In [21]:
def get_sentiment_score(start_date,end_date):
    
    filtered_df = df[(df.index >= start_date) & (df.index <= end_date)]
    
    if filtered_df.empty:
        return 'neutral', 0

    pos_score = filtered_df[ filtered_df['sentiment'] == 'positive']["scores"].sum()
    neg_score = filtered_df[ filtered_df['sentiment'] == 'negative']["scores"].sum()

    pos_score = 0 if pd.isna(pos_score) else pos_score
    neg_score = 0 if pd.isna(neg_score) else neg_score

    mode_sentiment = filtered_df["sentiment"].mode()
    
    # sum based on df.index
    if len(mode_sentiment) == 0:
        return 'neutral',0
    
    if len(mode_sentiment) > 1:
        return 'positive' if pos_score > neg_score else 'negative', pos_score - neg_score

    return mode_sentiment[0], pos_score - neg_score    



In [22]:
import yfinance as yf

ticker = 'AAPL'
start_date = '2018-05-17'
end_date = '2024-10-05'

stock_data = yf.download(ticker, start=start_date, end=end_date)
stock_data.index = pd.to_datetime(stock_data.index, format='%Y-%m-%d').date

[*********************100%***********************]  1 of 1 completed


In [23]:
stock_data['sentiment'] = None
stock_data['score'] = 0
stock_data.head()

,Open,High,Low,Close,Adj Close,Volume,sentiment,score
2018-05-17,47.000000,47.227501,46.590000,46.747501,44.388054,69176000,None,0
2018-05-18,46.797501,46.952499,46.532501,46.577499,44.226627,73190800,None,0
2018-05-21,47.000000,47.317501,46.727501,46.907501,44.539974,73603200,None,0
2018-05-22,47.095001,47.220001,46.695000,46.790001,44.428394,60962800,None,0
2018-05-23,46.587502,47.125000,46.439999,47.090000,44.713261,80233600,None,0


In [24]:
# stock_data.index.min() == df.index.min()


In [25]:
for i in range(len(stock_data)):
    if i == 0:
        start_date = stock_data.index[i] - timedelta(days=2)
        end_date = stock_data.index[i] - timedelta(days=1)
        
        sentiment, score = get_sentiment_score(start_date, end_date)
        
        # Use iloc for positional indexing
        stock_data.iloc[i, stock_data.columns.get_loc('sentiment')] = sentiment
        stock_data.iloc[i, stock_data.columns.get_loc('score')] = score
        
        print(stock_data.iloc[i, stock_data.columns.get_loc('sentiment')], stock_data.iloc[i, stock_data.columns.get_loc('score')])
    else:
        start_date = stock_data.index[i-1]
        end_date = stock_data.index[i] - timedelta(days=1)

        sentiment, score = get_sentiment_score(start_date, end_date)
        
        # Use iloc for positional indexing
        stock_data.iloc[i, stock_data.columns.get_loc('sentiment')] = sentiment
        stock_data.iloc[i, stock_data.columns.get_loc('score')] = score
        
        print(stock_data.iloc[i, stock_data.columns.get_loc('sentiment')], stock_data.iloc[i, stock_data.columns.get_loc('score')])


neutral 0
negative -0.07497315108776093
positive -0.5740335769951344
negative -0.2631111927330494
negative -0.24994856119155884
positive 0.1405654437839985
positive 1.095026083290577
positive 2.0848943404853344
negative -1.868009865283966
positive 0.5523335412144661
negative -2.423324916511774
positive -1.3566753193736076
positive 5.684445179998875
positive -0.9509128499776125
positive 2.646383337676525
neutral 0.0
negative -4.207538489252329
positive 0.4284137040376663
negative -0.7521094679832458
negative -4.003694824874401
negative -0.5445416569709778
positive 1.5846631824970245
positive 0.10612589865922928
negative -2.5142799392342567
positive -1.0286568477749825
positive 0.011355206370353699
negative -1.1027515530586243
positive 1.2216513380408287
positive 1.8579614758491516
negative -1.3935932219028473
positive 0.049806058406829834
positive 0.7404717914760113
positive 0.06478121876716614
negative -1.0272888988256454
positive 0.17440257966518402
positive 0.07105352729558945
negati

C:\Users\Ishit\AppData\Local\Temp\ipykernel_21888\2435401705.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.07497315108776093' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  stock_data.iloc[i, stock_data.columns.get_loc('score')] = score


negative -0.8728862889111042
negative -0.20292039960622787
positive 1.0286507681012154
positive 0.4543521851301193
positive 0.4598182439804077
positive 1.3370745740830898
negative -4.3806955721229315
positive -0.0023027658462524414
positive 0.8589947521686554
positive 0.15692367404699326
negative -7.539572771638632
positive -1.4837455190718174
positive 7.282264672219753
positive -2.6304693650454283
positive 12.71701941639185
positive -0.6250026561319828
negative -8.33291918411851
negative -1.9071625880897045
positive 0.3519153483211994
positive 0.15981373190879822
negative -0.8368090689182281
positive 0.6554244831204414
negative -5.484301447868347
positive 0.6775418370962143
positive 9.593384250998497
negative -1.7087881863117218
negative -2.8082278668880463
positive 3.073518555611372
positive 0.2502877861261368
negative -4.771034926176071
negative -5.477420100942254
positive -0.4532548002898693
positive 0.8891117870807648
positive 0.14347074180841446
positive 6.030578028410673
negativ

In [35]:
stock_data[:10]

,Open,High,Low,Close,Adj Close,Volume,sentiment,score
2021-04-19,133.509995,135.470001,133.339996,134.839996,132.196274,94264200,neutral,0.000000
2021-04-20,135.020004,135.529999,131.809998,133.110001,130.500198,94812300,positive,0.511896
2021-04-21,132.360001,133.750000,131.300003,133.500000,130.882523,68847100,positive,3.324788
2021-04-22,133.039993,134.149994,131.410004,131.940002,129.353134,84566500,positive,-0.032559
2021-04-23,132.160004,135.119995,132.160004,134.320007,131.686493,78657500,positive,-0.419344
2021-04-26,134.830002,135.059998,133.559998,134.720001,132.078644,66905100,positive,1.533566
2021-04-27,135.009995,135.410004,134.110001,134.389999,131.755096,66015800,negative,0.098440
2021-04-28,134.309998,135.020004,133.080002,133.580002,130.960983,107760100,positive,0.823338
2021-04-29,136.470001,137.070007,132.449997,133.479996,130.862930,151101000,positive,2.051200
2021-04-30,131.779999,133.559998,131.070007,131.460007,128.882553,109839500,positive,1.219282


In [160]:
stock_data.loc[1,'sentiment'] , stock_data.loc[0,'score']

('positive', 0)

In [26]:
stock_data.sentiment.value_counts()

sentiment
positive    1243
negative     347
neutral       17
Name: count, dtype: int64

In [27]:
stock_data.to_csv('stock_data_large.csv')